In [1]:
import pandas as pd
import numpy as np
from metrics import TCS
from sklearn.metrics import r2_score,mean_absolute_error,mean_squared_error
import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio
pio.templates.default = 'plotly_white'
plot_template = dict(
    layout=go.Layout({
        'font_size': 8,
        'xaxis_title_font_size': 8,
        'yaxis_title_font_size': 8,
        }   
))
hydro_stations = ['Tangnaihai','Guide','Xunhua']
hydrostation_abbrs = {'Tangnaihai':'TNH','Guide':'GD','Xunhua':'XH'}
hydrostation_channel = {'Tangnaihai':'3','Guide':'14','Xunhua':'9'}


In [2]:
pred_years = np.arange(2015,2020)
index = pd.date_range(start=f'{pred_years[0]}-01-01',end=f'{pred_years[-1]}-12-31',freq='MS')
for hydro_station in hydro_stations:
    hydro_station_abbr = hydrostation_abbrs[hydro_station]
    hydro_station_channel = hydrostation_channel[hydro_station]
    pred_df_swatp_mete_simyr = pd.DataFrame(index=index,columns=['obs','pred'])
    pred_df_swatp_mete_simyr.index.name='date'
    pred_df_swatp_hydro_simyr = pd.DataFrame(index=index,columns=['obs','pred'])
    pred_df_swatp_hydro_simyr.index.name='date'
    
    obs_df = pd.read_csv(f'../data/{hydro_station.lower()}_natural_monthly_flow.csv',index_col=['date'],parse_dates=['date'])
    obs_his = obs_df.copy().loc[:f'{pred_years[0]-1}-12-31']
    obs_his_monthly_mean = obs_his.groupby(obs_his.index.month)['flow(m^3/s)'].mean()
    # Create a new DataFrame with monthly average flow for 2015
    df_monthly_avg = pd.DataFrame(index=index)
    # Replicate the monthly average flow for years 2015-2019
    for year in range(pred_years[0],pred_years[-1]+1):
        for month in range(1,13):   
            df_monthly_avg.loc[f'{year}-{month}-01','flow(m^3/s)'] = obs_his_monthly_mean[month]

    
    for pred_year in pred_years:
        date_range = pd.date_range(start=f'{pred_year}-01-01',end=f'{pred_year}-12-31',freq='MS')
        set_index = pd.DatetimeIndex(date_range)
        pred_data_swatp_mete_simyr = pd.read_csv(f'../result/SWATPlusPredUsingMeteSimYearData/Channel_{hydro_station_channel}_Monthly_River-Flow_{hydro_station_abbr}_pred{pred_year}.csv',index_col=['Date'],parse_dates=['Date'])
        pred_data_swatp_hydro_simyr = pd.read_csv(f'../result/SWATPlusPredUsingHydroSimYearData/Channel_{hydro_station_channel}_Monthly_River-Flow_{hydro_station_abbr}_pred{pred_year}.csv',index_col=['Date'],parse_dates=['Date'])
        
        pred_df_swatp_mete_simyr.loc[f'{pred_year}-01-01':f'{pred_year}-12-31','pred'] = pred_data_swatp_mete_simyr.loc[f'{pred_year}-01-01':f'{pred_year}-12-31','Value'].values
        pred_df_swatp_mete_simyr.loc[f'{pred_year}-01-01':f'{pred_year}-12-31','obs'] = obs_df.loc[f'{pred_year}-01-01':f'{pred_year}-12-31','flow(m^3/s)'].values
        pred_df_swatp_hydro_simyr.loc[f'{pred_year}-01-01':f'{pred_year}-12-31','pred'] = pred_data_swatp_hydro_simyr.loc[f'{pred_year}-01-01':f'{pred_year}-12-31','Value'].values
        pred_df_swatp_hydro_simyr.loc[f'{pred_year}-01-01':f'{pred_year}-12-31','obs'] = obs_df.loc[f'{pred_year}-01-01':f'{pred_year}-12-31','flow(m^3/s)'].values
       
        
    pred_df_swatp_mete_simyr.to_csv(f'../result/SWATPlusPredUsingMeteSimYearData/{hydro_station}_SWATPlus_pred_obs_{pred_years[0]}_{pred_years[-1]}.csv')
    pred_df_swatp_hydro_simyr.to_csv(f'../result/SWATPlusPredUsingHydroSimYearData/{hydro_station}_SWATPlus_pred_obs_{pred_years[0]}_{pred_years[-1]}.csv')